In [2]:
! pip install langchain langchain-community openai tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference youtube-transcript-api pytube sentence-transformers


In [9]:
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata

In [10]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key
pc = Pinecone()
pc.create_index(
    name="raggl", dimension=1024, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '6d84a2ef134d682f9930b557b2cf7433', 'Date': 'Sun, 25 Aug 2024 19:20:37 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [4]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Imran Khan',
  'review': 'Clear lectures and easier to understand. If you focus well in class, you can easily score 4 GPA, but grading is strict.',
  'subject': 'DEVOPS',
  'stars': 4},
 {'professor': 'Ms. Sana Ali',
  'review': 'Explains concepts thoroughly with real-world examples, but assignments are tough and require a lot of effort.',
  'subject': 'DEVOPS',
  'stars': 3},
 {'professor': 'Mr. Asif Malik',
  'review': 'Great teaching style with interactive sessions. However, his quizzes can be tricky.',
  'subject': 'DEVOPS',
  'stars': 4},
 {'professor': 'Dr. Rabia Qureshi',
  'review': 'Focuses on practical knowledge and hands-on labs. Very helpful during office hours.',
  'subject': 'DEVOPS',
  'stars': 5},
 {'professor': 'Mr. Usman Tariq',
  'review': "Engaging lectures but sometimes goes too fast. Make sure to ask questions if you don't understand.",
  'subject': 'DEVOPS',
  'stars': 3},
 {'professor': 'Ms. Zainab Farooq',
  'review': 'Provides a lot of reso

In [5]:
hf_embeddings = HuggingFaceEmbeddings(model_name="dunzhang/stella_en_1.5B_v5")

def get_embedding(text):
    return hf_embeddings.embed_query(text)

with open('reviews.json', 'r') as file:
    data = json.load(file)

processed_data = []

for review in data['reviews']:
    embedding = get_embedding(review["review"])

    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

print(processed_data)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been 

[{'values': [0.0617617629468441, 0.47455650568008423, 0.3223210275173187, -0.2973456084728241, 0.22459451854228973, -0.016067052260041237, -0.11516300588846207, -0.09934438765048981, 0.6810036897659302, 0.2584541440010071, -0.6112951040267944, -0.02126239612698555, 0.5870991349220276, 0.330180823802948, 0.15199653804302216, 0.17775368690490723, -0.42037713527679443, -0.3923339545726776, -0.10329395532608032, -0.20605000853538513, 0.0314592570066452, -0.39558079838752747, -0.38036268949508667, -0.45681193470954895, -0.5404291749000549, -0.25958529114723206, -0.16974180936813354, -0.38259392976760864, 0.4259779155254364, -0.07683580368757248, -0.4975457787513733, -0.14521488547325134, -0.2605755031108856, -0.14170879125595093, 0.39805543422698975, 0.25863131880760193, 0.49139904975891113, 0.465576708316803, 0.03989359736442566, 0.5322813391685486, 0.2822026014328003, -0.36226755380630493, 0.5543739199638367, 0.18884491920471191, -0.32796213030815125, -0.37425360083580017, -0.022281900048

In [6]:
processed_data[0]

{'values': [0.0617617629468441,
  0.47455650568008423,
  0.3223210275173187,
  -0.2973456084728241,
  0.22459451854228973,
  -0.016067052260041237,
  -0.11516300588846207,
  -0.09934438765048981,
  0.6810036897659302,
  0.2584541440010071,
  -0.6112951040267944,
  -0.02126239612698555,
  0.5870991349220276,
  0.330180823802948,
  0.15199653804302216,
  0.17775368690490723,
  -0.42037713527679443,
  -0.3923339545726776,
  -0.10329395532608032,
  -0.20605000853538513,
  0.0314592570066452,
  -0.39558079838752747,
  -0.38036268949508667,
  -0.45681193470954895,
  -0.5404291749000549,
  -0.25958529114723206,
  -0.16974180936813354,
  -0.38259392976760864,
  0.4259779155254364,
  -0.07683580368757248,
  -0.4975457787513733,
  -0.14521488547325134,
  -0.2605755031108856,
  -0.14170879125595093,
  0.39805543422698975,
  0.25863131880760193,
  0.49139904975891113,
  0.465576708316803,
  0.03989359736442566,
  0.5322813391685486,
  0.2822026014328003,
  -0.36226755380630493,
  0.554373919963836

In [11]:
pc = Pinecone()
index = pc.Index("raggl")
index.upsert(
    vectors=processed_data,
    namespace="reviews-1"
)

{'upserted_count': 48}

In [12]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}